# NYT Crossword Scraping
#### Written by N. Rauchle 

In [102]:
#! pip install webdriver-manager
#! pip install selenium
#! pip install reportlab
#! pip install frontend

## For under construction
# ! pip install fitz
# ! pip install tools
# ! pip uninstall pymupdf
# ! pip install html-testRunner
# ! pip install JPype1


In [87]:
# Install packages
import selenium 
import requests
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains # provides methods for performing complex user interaction
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.support.ui import WebDriverWait
from PIL import Image
from reportlab.pdfgen import canvas
from datetime import datetime

import time # time.sleep(sec) is used throughout the script to allow the webpages to fully load before moving on to the next command

# Tested for Under Construction
# if not os.path.exists('static'):
    # os.makedirs('static')
#from io import BytesIO
#import fitz
#from fitz.utils import Rect
#import jpype


In [88]:
# Define Variables
folder_location = '/Users/gcano01/NR'
driver_path = Service(folder_location + '/chromedriver')
chrome_profile_path = '/Users/gcano01/Library/Application Support/Google/Chrome/Profile 1'

crossword_url = "https://www.nytimes.com/crosswords"
username = "thomas_raeuchle@yahoo.com"
password = "ny828482"

## Step 1
##### Note: if you are running this script for the first time, start here. Otherwise, jump to step 2
#### Open the NYT games login page and log in. 

In [30]:
options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features")
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument("--headless")
options.add_argument('user-data-dir=' + chrome_profile_path)
driver = webdriver.Chrome(options=options, service = driver_path)
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
driver.execute_cdp_cmd('Network.setUserAgentOverride', {"userAgent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.53 Safari/537.36'})

driver.get(crossword_url)
time.sleep(3)

# Select login
login_prompt = driver.find_element(By.XPATH, '/html/body/header/div[1]/div[2]/a[2]').click()

# Send username info
username_field = driver.find_element(By.XPATH, '//*[@id="email"]')
username_field.send_keys(username)

time.sleep(3)
driver.find_element(By.XPATH, '/html/body/div/div/div/div/form/div/div[4]/button').click()
time.sleep(3)

# Send password info
# Enter password 
password_field = driver.find_element(By.XPATH, '//*[@id="password"]')
password_field.send_keys(password)
time.sleep(3)
driver.find_element(By.XPATH, '/html/body/div/div/div/form/div/div[2]/button').click()

## Stop here and complete CAPTCHA authentication ##

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/header/div[1]/div[2]/a[2]"}
  (Session info: chrome=111.0.5563.64)
Stacktrace:
0   chromedriver                        0x0000000100924428 chromedriver + 4899880
1   chromedriver                        0x00000001008a1a23 chromedriver + 4364835
2   chromedriver                        0x00000001004ebbf6 chromedriver + 474102
3   chromedriver                        0x000000010052f4f0 chromedriver + 750832
4   chromedriver                        0x000000010052f751 chromedriver + 751441
5   chromedriver                        0x0000000100573834 chromedriver + 1030196
6   chromedriver                        0x000000010055558d chromedriver + 906637
7   chromedriver                        0x0000000100570b5b chromedriver + 1018715
8   chromedriver                        0x0000000100555333 chromedriver + 906035
9   chromedriver                        0x000000010051f55f chromedriver

##### !**Stop here and complete CAPTCHA verification**!
##### Note: efforts to bypass CAPTCHA verification can be found at the bottom of the script under section titled "Under Construction". Because of Chrome security updates, it's become increasingly difficult to convince Chrome that you're not a robot. 

## STEP 2
#### Once you've logged in your credentials should be saved the when the driver relaunches the url. This will bring you to the crossword homepage with your login information preserved. 

##### Note: If you want to run this chunk multiple times, you must completely close out of the window previously opened by script.

In [103]:
# Set up webdriver
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument('user-data-dir=' + chrome_profile_path)
options.add_argument("--disable-gpu") # For screenshot quality
driver = webdriver.Chrome(options=options, service = driver_path)
driver.execute_cdp_cmd('Network.setUserAgentOverride', {"userAgent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.53 Safari/537.36'})
wait = WebDriverWait(driver, 10) # define wait time for driver - up to 10 seconds

# Re-open crossword homepage
driver.get(crossword_url)

# Select menu and naviagte to the archive page
menu_button = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="js-nav-burger"]')))
menu_button.click()

archive_button = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="js-nav-drawer"]/nav/div[9]/a')))
archive_button.click()

# The print button is only available when cursor hovers over puzzle so we'll find the element, tell our cursor to hover,
# and once the print button becomes interactable, we'll select it. 
for i in range(2,32):
    # Locate the element that triggers the print button to appear
    hover_element = wait.until(EC.visibility_of_element_located((By.XPATH, f'//*[@id="crossword-archive-container"]/div/div/div[3]/div[2]/div/div[2]/div[{i}]/div/a')))
                                                                        
    # Use ActionChains to move the cursor to the location where the button will appear
    action = ActionChains(driver)
    action.move_to_element(hover_element).perform()
    
    # Wait for the button that is triggered by hover action to appear then select it
    button_element = wait.until(EC.visibility_of_element_located((By.XPATH, f'//*[@id="crossword-archive-container"]/div/div/div[3]/div[2]/div/div[2]/div[{i}]/div/div')))
    button_element.click()

    # Scrape date information to set naming variables
    # Pause for page to load
    time.sleep(1)

    ## SET NAMING VARIABLES -----
    
    # Get the date from the puzzle options
    day_text = driver.find_element(By.XPATH, '//*[@id="pzm-print"]/div[2]/div/div[1]/div[2]').text
    print(day_text)

    # Find the day of the week from the date 
    day = datetime.strptime(day_text, '%B %d, %Y').strftime('%a').lower()
    print(day)

    # Set the file_name 
    file_name =  day_text.replace(" ", "_").replace(",", "").lower()
    print(file_name)

    png_name = f'{file_name}.png'
    pdf_file_name = f'{folder_location}/nyt_crossword/{day}/{day}_{file_name}.pdf'
 
    # Create a day of the week folder if it doesn't yet exist 
    if not os.path.exists(f'{folder_location}/nyt_crossword/{day}/'):
        os.makedirs(f'{folder_location}/nyt_crossword/{day}/')
    ## --------

    # Select print. This will primpt the crossword PDF to open in a new tab
    print_element = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="pzm-print"]/div[2]/div/div[2]/div[3]/button')))
    print_element.click()
    wait.until(EC.number_of_windows_to_be(2))

    # Switch the webdrive to the new tab to be able to interact there.
    current_window = driver.current_window_handle
    for window_handle in driver.window_handles:
        if window_handle != current_window:
            driver.switch_to.window(window_handle)
            break 

    # NYT website isn't preserving cookies when opening a new tab and is recognizing the driver as a bot :( therefore blocking dowloads
    # Using selenium to take screenshots instead and save as pdf. 
    time.sleep(2)

    #driver.execute_script('document.body.style.zoom = "80%"')

    #driver.set_window_size(1600, pdf_height)
    #time.sleep(2)
    #driver.set_window_size(1400, 900)
    #driver.send_keys(Keys.CONTROL + "-")
    driver.save_screenshot(file_name)
    image = Image.open(file_name).convert("L")
    image.save(png_name)

    
    #pdf_file_name = f'/Users/gcano01/NR/nyt_crossword/{day}_{file_name}.pdf'
    pdf = canvas.Canvas(pdf_file_name, image.size) # pagesize=png_name.size was imagine.size
    print(pdf_file_name) # TO DO: figure out why canvas isn't saving file all lowercase.
    pdf.filename = pdf_file_name
    pdf.drawImage(png_name, 0, 0)
    pdf.save()
    
    # Close the current tab and switch to original handle
    driver.close()
    driver.switch_to.window(driver.window_handles[0])
    
    # close the print menu
    close_element = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="pzm-print"]/div[1]')))
    close_element.click()
    

    # Delete the PNG file
    os.remove(png_name)
    os.remove(file_name)




TimeoutException: Message: 


In [ ]:
# Mon 
# //*[@id="crossword-archive-container"]/div/div/div[3]/div[2]/div/div[2]/div[7]/div/div/a
# //*[@id="crossword-archive-container"]/div/div/div[3]/div[2]/div/div[2]/div[14]/div/div/a
# Tue
# //*[@id="crossword-archive-container"]/div/div/div[3]/div[2]/div/div[2]/div[8]/div/div/a

# Wed
# //*[@id="crossword-archive-container"]/div/div/div[3]/div[2]/div/div[2]/div[9]/div/div/a
# Thu
# //*[@id="crossword-archive-container"]/div/div/div[3]/div[2]/div/div[2]/div[10]/div/div/a
# Fri
# //*[@id="crossword-archive-container"]/div/div/div[3]/div[2]/div/div[2]/div[11]/div/div/a
# Sat
# //*[@id="crossword-archive-container"]/div/div/div[3]/div[2]/div/div[2]/div[12]/div/div/a
# Sun
# //*[@id="crossword-archive-container"]/div/div/div[3]/div[2]/div/div[2]/div[6]/div/div/a

In [ ]:
total_height = 1060
total_width = 826
viewport_height = 1000


# Set the initial scroll position
scroll_pos = 0

# Create a list to store the screenshots
screenshots = []

img = Image.new('RGB',(total_width, total_height), (255, 255, 255))
    
while scroll_pos < total_height:
    driver.execute_script(f'window.scrollTo(0, {scroll_pos});')
    img_filename = f'screenshot_{scroll_pos}.png'
    driver.save_screenshot(img_filename)
    img_frag = Image.open(img_filename)
    img.paste(img_frag, (0, scroll_pos))
    scroll_pos += img_frag.size[1]

# Save the full page screenshot of the PDF document
img.save('full_page_screenshot.png')


# Merge the screenshots into a single image
merged_screenshot = Image.new('RGB', (screenshots[0].width, total_height))
current_height = 0
for screenshot in screenshots:
    merged_screenshot.paste(screenshot, (0, current_height))
    current_height += screenshot.height

# Save the merged screenshot
merged_screenshot.save("full_page_screenshot.png")


# Loop through the entire page and take screenshots
while scroll_pos < total_height:
    # Set the scroll position
    driver.execute_script(f"window.scrollTo(0, {scroll_pos})")
    time.sleep(1)
    
    # Take a screenshot of the current viewport
    screenshot = driver.get_screenshot_as_png()
    screenshot = Image.open(BytesIO(screenshot))
    
    # Append the screenshot to the list
    screenshots.append(screenshot)
    

# Find the element we want to screenshit 
pdf_element = driver.find_element(By.CSS_SELECTOR, 'body > embed')
location = pdf_element.location
size = pdf_element.size
print(location)
print(size)

x = location['x']
y = location['y']

width = size['width']
height = size['height']
pdf_rect = Rect(x, y, x+width, y+height)
pdf = fitz.open(pdf_file_name)
page = pdf[0]
page_pixmap = page.get_pixmap(matrix=fitz.Matrix(1, 1), clip = pdf_rect)
print(page_pixmap)
page_pixmap.save(png_name)

In [17]:
print(page_pixmap)

Pixmap(DeviceRGB, IRect(0, 0, 1400, 742), 0)


In [ ]:
# Turn PDF Scrape into a function 


In [ ]:
x.is_displayed()
x.is_selected()
x.screenshot_as_base64()

In [ ]:
# XPATHS
# Sunday
# Full XPATH
# /html/body/div[1]/div/div[3]/div/div/div/div[3]/div[2]/div/div[2]/div[6]/div/a/div/div[1]/div[2]/span
# /html/body/div[1]/div/div[3]/div/div/div/div[3]/div[2]/div/div[2]/div[13]/div/a/div/div[1]/div[2]/span
# /html/body/div[1]/div/div[3]/div/div/div/div[3]/div[2]/div/div[2]/div[20]/div/a/div/div[1]/div[2]/span
# /html/body/div[1]/div/div[3]/div/div/div/div[3]/div[2]/div/div[2]/div[27]/div/a/div/div[1]/div[2]/span
# //*[@id="crossword-archive-container"]/div/div/div[3]/div[2]/div/div[2]/div[2]/div/div/a

# Monday 





# Simple XPATH SUNDAY
# //*[@id="crossword-archive-container"]/div/div/div[3]/div[2]/div/div[2]/div[2]/div/div/a
# //*[@id="crossword-archive-container"]/div/div/div[3]/div[2]/div/div[2]/div[9]/div/div/a

# //*[@id="crossword-archive-container"]/div/div/div[3]/div[2]/div/div[2]/div[6]/div/div/a
# //*[@id="crossword-archive-container"]/div/div/div[3]/div[2]/div/div[2]/div[13]/div/div/a

# SIMPLE XPATH MONDAY 
# //*[@id="crossword-archive-container"]/div/div/div[3]/div[2]/div/div[2]/div[7]/div/div/a
# //*[@id="crossword-archive-container"]/div/div/div[3]/div[2]/div/div[2]/div[14]/div/div/a

# From another both sunday = 1
# //*[@id="crossword-archive-container"]/div/div/div[3]/div[2]/div/div[2]/div[2]/div/a/div/div[1]/div[2]/span

# Print 
# //*[@id="crossword-archive-container"]/div/div/div[3]/div[2]/div/div[2]/div[6]/div/div/a/div


## Under Construction

### Resources
#### Prevent selenium detection:
 - https://stackoverflow.com/questions/58872451/how-can-i-bypass-the-google-captcha-with-selenium-and-python
 - https://stackoverflow.com/questions/53039551/selenium-webdriver-modifying-navigator-webdriver-flag-to-prevent-selenium-detec/53040904#53040904
 - https://stackoverflow.com/questions/66199053/evade-detection-of-selenium-automation/66200459#66200459
#### Changing user agent: 
 - https://stackoverflow.com/questions/49565042/way-to-change-google-chrome-user-agent-in-selenium/49565254#49565254
#### Information on CAPTCHA: 
 - https://stackoverflow.com/questions/55501524/how-does-recaptcha-3-know-im-using-selenium-chromedriver/55502835#55502835
#### Saving Cookies to open un-clean chrome window 
 - https://stackoverflow.com/questions/15058462/how-to-save-and-load-cookies-using-python-selenium-webdriver/15058521#15058521 
#### Download PDF 
 - https://stackoverflow.com/questions/66219052/cannot-download-pdf-using-python-requests
 - https://stackoverflow.com/questions/20361249/http-error-401-authorization-required-while-downloading-a-file-from-https-websi

In [ ]:
# Use headless chrome to access NYT

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

# set up Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")

# set up Chrome driver
driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)

# navigate to a website
url = "https://www.example.com"
driver.get(url)

# print the page title
print(driver.title)

# close the browser
driver.quit()

In [ ]:
# Download PDF
# Download crossword 
pdf_url = driver.current_url
time.sleep(5)

#user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.53 Safari/537.36'
#headers = {'User-Agent': user_agent}

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.5563.64 Safari/537.36'}

response = requests.get(pdf_url, headers=headers, stream = True)

#import urllib.request
#urllib.request.urlretrieve(pdf_url, '/Users/gcano01/NR/nyt_crossword/filename.pdf')

print(pdf_url)
pdf_content = response.content
print(pdf_content)

with open(os.path.join('/Users/gcano01/NR/nyt_crossword/', day_text) + '.pdf', "wb") as f:
    for chunk in response.iter_content(chunk_size = 1024):
        f.write(chunk)
        